In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggCons.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggCor.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggMeanInf.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggMeanInf0.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggCons0.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/yAggThr.RData")

load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCons.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCor.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC0.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeThr.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCons0.RData")

suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))

In [6]:
txpDiff <- setdiff(treeMC0$tip.label,treeMC$tip.label)
length(txpDiff)

[1] 6547

In [5]:
# infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
# infReps <- abind::abind(as.list(infReps), along = 3)
# lfc <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])
# lfcMet <- abs(lfc/mcols(yAggMeanInf0)[["meanInfRV"]])

# wlfcDiff <- sum(lfcMet[txpDiff])
# wlfcDiff
# l <- length(treeCons$tip)
wlfcDiff <- 0
txpDiff <- NULL
length(txpDiff)

[1] 0

### Consensus Tree

In [6]:
dfLFC <- data.frame(matrix(nrow=0, ncol=3))
dCuts <- list()
colnames(dfLFC) <- c("Type", "Value", "nCuts")
l <- length(treeCons$tip)
lengths <- sapply(Descendants(treeCons, seq(length(treeCons$tip)+treeCons$Nnode)), length)
infReps <- assays(yAggCons)[grep("infRep", assayNames(yAggCons))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCons)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggCons)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCons))
val <- findMaxSum(treeCons, lfcMet, l+1, lengths)
optLfcMet <- globArr
cuts <- findCuts(treeCons, optLfcMet, lfcMet, l+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="Consensus", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCuts[["Consensus"]] <- cuts

### Correlation Tree

In [8]:
l==length(treeCor$tip)
infReps <- assays(yAggCor)[grep("infRep", assayNames(yAggCor))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCor)[["condition"]])
lengths <- sapply(Descendants(treeCor, seq(length(treeCor$tip)+treeCor$Nnode)), length)
lfcMet <- abs(lfc/mcols(yAggCor)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCor))
val <- findMaxSum(treeCor, lfcMet, l+1, lengths)
optLfcMet <- globArr
cuts <- findCuts(treeCor, optLfcMet, lfcMet, l+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="Anti-Correlation", Value=val + wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCuts[["Anti-Correlation"]] <- cuts

[1] TRUE

### Mean Inf

In [12]:
l==length(treeMeanInf$tip)
infReps <- assays(yAggMeanInf)[grep("infRep", assayNames(yAggMeanInf))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf)[["condition"]])
lfcMet <- abs(lfc/mcols(yAggMeanInf)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggMeanInf))
lengths <- sapply(Descendants(treeMeanInf, seq(length(treeMeanInf$tip)+treeMeanInf$Nnode)), length)
val <- findMaxSum(treeMeanInf, lfcMet, l+1, lengths)
optLfcMet <- globArr
cuts <- findCuts(treeMeanInf, optLfcMet, lfcMet, l+1,lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCuts[["MeanInf"]] <- cuts

[1] TRUE

In [13]:
dfLFC

Type,Value,nCuts
<chr>,<dbl>,<int>
Consensus,850717.1,41246
Anti-Correlation,708478.1,84082
MeanInf,615488.0,58263


### Mean Inf Thr0

In [14]:
l==length(treeMeanInf0$tip)
infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggMeanInf0)[["meanInfRV"]])
lengths <- sapply(Descendants(treeMeanInf0, seq(length(treeMeanInf0$tip)+treeMeanInf0$Nnode)), length)
globArr <- rep(-100, nrow(yAggMeanInf0))
val <- findMaxSum(treeMeanInf0, lfcMet, length(treeMeanInf0$tip)+1, lengths)
optLfcMet <- globArr
print(val)
cuts <- findCuts(treeMeanInf0, optLfcMet, lfcMet, length(treeMeanInf0$tip)+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf0", Value=val, nCuts = length(cuts)))
dCuts[["MeanInf0"]] <- cuts

[1] TRUE

[1] 1018883


In [15]:
print(c(val, length(cuts)))

[1] 1018883   30792


### Consensus Thr

In [18]:
l==length(treeThr$tip)
infReps <- assays(yAggThr)[grep("infRep", assayNames(yAggThr))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggThr)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggThr)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggThr))
lengths <- sapply(Descendants(treeThr, seq(length(treeThr$tip)+treeThr$Nnode)), length)
val <- findMaxSum(treeThr, lfcMet, l+1,lengths)
optLfcMet <- globArr
cuts <- findCuts(treeThr, optLfcMet, lfcMet, l+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="Cons_Thr", Value=val+wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCuts[["Cons_Thr"]] <- cuts

[1] TRUE

In [19]:
dfLFC

Type,Value,nCuts
<chr>,<dbl>,<int>
Consensus,850717.1,41246
Anti-Correlation,708478.1,84082
MeanInf,615488.0,58263
MeanInf0,1018883.0,30792
Cons_Thr,330390.6,1


In [24]:
l==length(treeCons0$tip)
infReps <- assays(yAggCons0)[grep("infRep", assayNames(yAggCons0))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCons0)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggCons0)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCons0))
lengths <- sapply(Descendants(treeCons0, seq(length(treeCons0$tip)+treeCons0$Nnode)), length)
val <- findMaxSum(treeCons0, lfcMet, l+1,lengths)
optLfcMet <- globArr
cuts <- findCuts(treeCons0, optLfcMet, lfcMet, l+1, lengths)
dfLFC <- rbind(dfLFC, data.frame(Type="Consensus0", Value=val+wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCuts[["Consensus0"]] <- cuts

[1] TRUE

In [25]:
dfLFC

Type,Value,nCuts
<chr>,<dbl>,<int>
Consensus,850717.1,41246
Anti-Correlation,708478.1,84082
MeanInf,615488.0,58263
MeanInf0,1018883.0,30792
Cons_Thr,330390.6,1
Consensus0,1010767.3,31331


In [26]:
save(dCuts, file="environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/updated/comp_trees/lfcCuts_len.RData")

#### Consensus Tree

In [21]:
dfLFC <- data.frame(matrix(nrow=0, ncol=3))
dCutsNL <- list()
colnames(dfLFC) <- c("Type", "Value", "nCuts")
l <- length(treeCons$tip)
infReps <- assays(yAggCons)[grep("infRep", assayNames(yAggCons))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCons)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggCons)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCons))
val <- findMaxSum(treeCons, lfcMet, l+1)
optLfcMet <- globArr
cuts <- findCuts(treeCons, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="Consensus", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCutsNL[["Consensus"]] <- cuts

#### Correlation

In [28]:
l==length(treeCor$tip)
infReps <- assays(yAggCor)[grep("infRep", assayNames(yAggCor))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggCor)[["condition"]])
lfcMet <- abs(lfc/mcols(yAggCor)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggCor))
val <- findMaxSum(treeCor, lfcMet, l+1)
optLfcMet <- globArr
cuts <- findCuts(treeCor, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="Anti-Correlation", Value=val + wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCutsNL[["Anti-Correlation"]] <- cuts

[1] TRUE

#### MeanInf

In [32]:
l==length(treeMC$tip)
infReps <- assays(yAggMeanInf)[grep("infRep", assayNames(yAggMeanInf))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf)[["condition"]])
lfcMet <- abs(lfc/mcols(yAggMeanInf)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggMeanInf))
val <- findMaxSum(treeMC, lfcMet, l+1)
optLfcMet <- globArr
cuts <- findCuts(treeMC, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf", Value=val+wlfcDiff, nCuts = length(cuts)+length(txpDiff)))
dCutsNL[["MeanInf"]] <- cuts

[1] TRUE

#### MeanInf0

In [39]:
# l==length(treeMC0$tip)
infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggMeanInf0)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggMeanInf0))
val <- findMaxSum(treeMC0, lfcMet, length(treeMC0$tip)+1)
optLfcMet <- globArr
print(val)
cuts <- findCuts(treeMC0, optLfcMet, lfcMet, length(treeMC0$tip)+1)
dfLFC <- rbind(dfLFC, data.frame(Type="MeanInf0", Value=val, nCuts = length(cuts)))
dCutsNL[["MeanInf0"]] <- cuts


[1] 224250.6


In [40]:
dfLFC

Type,Value,nCuts
<chr>,<dbl>,<int>
Consensus,210766.7,93762
Anti-Correlation,189507.6,118417
MeanInf,210347.4,96183
MeanInf0,224250.6,86958


#### Thr

In [47]:
l==length(treeThr$tip)
infReps <- assays(yAggThrNS)[grep("infRep", assayNames(yAggThrNS))]
infReps <- abind::abind(as.list(infReps), along = 3)
lfc <- getLog2FC(infReps, colData(yAggThrNS)[["condition"]])

lfcMet <- abs(lfc/mcols(yAggThrNS)[["meanInfRV"]])
globArr <- rep(-100, nrow(yAggThrNS))
val <- findMaxSum(treeThr, lfcMet, l+1)
optLfcMet <- globArr
cuts <- findCuts(treeThr, optLfcMet, lfcMet, l+1)
dfLFC <- rbind(dfLFC, data.frame(Type="Cons_Thr", Value=val+wlfcDiff, nCuts = length(cuts) + length(txpDiff)))
dCutsNL[["Cons_Thr"]] <- cuts

[1] TRUE

In [49]:
save(dCutsNL, file="environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/updated/comp_trees/lfcCuts_nolen.RData")

In [48]:
length(dCutsNL)

[1] 5